In [ ]:
import os
import sys
import h5py
import argparse
import numpy as np
from collections import Counter
import xml.etree.ElementTree as et 

# Keras imports
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.layers import Input, Dense, Dropout, AlphaDropout, BatchNormalization, Activation

import matplotlib
matplotlib.use('Agg') # Need this to write images from the GSA servers.  Order matters:
import matplotlib.pyplot as plt # First import matplotlib, then use Agg, then import plt

# IPython imports
from IPython.display import Image

# ml4h Imports
from ml4h.arguments import parse_args
from ml4h.tensor_writer_ukbb import write_tensors
from ml4h.tensor_generators import test_train_valid_tensor_generators
from ml4h.recipes import train_multimodal_multitask, test_multimodal_multitask
from ml4h.models import make_multimodal_multitask_model, train_model_from_generators

%matplotlib inline

In [ ]:
os.listdir('/mnt/disks/ecg-rest-and-bike-with-trend-tensors/2019-08-13/')[0]

In [ ]:
sys.argv = ['train', 
            '--tensors', '/mnt/disks/ecg-rest-and-bike-with-trend-tensors/2019-08-13/', 
            '--input_tensors', 'ecg_bike_recovery', 
            '--output_tensors','ecg-bike-max-pred-hr-no0', 'ecg-bike-max-hr-no0', 'ecg-bike-resting-hr-no0',
            '--batch_size', '64', 
            '--epochs', '20',
            '--training_steps', '5',
            '--inspect_model',
            '--u_connect',
            '--learning_rate', '0.01',
            '--id', 'ecg_bike_recovery_labeler',
            '--output_folder', '/home/ndiamant/ml/notebooks/recipes_output/ecg_sketch'
           ]
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,  args.tensor_maps_out,  args.tensors, args.batch_size,   args.valid_ratio, args.test_ratio, args.test_modulo, args.balance_csvs)

In [ ]:
model = make_multimodal_multitask_model(**args.__dict__)


In [ ]:
# model.predict_generator(generate_test, steps=1)

In [ ]:
model = train_model_from_generators(model, generate_train, generate_valid, args.training_steps,
                                        args.validation_steps, args.batch_size, args.epochs, args.patience,
                                        args.output_folder, args.id, args.inspect_model, args.inspect_show_labels)

In [ ]:
step = generate_test.__next__()
step[1].keys()
print(step[1]['output_bike_max_pred_hr_continuous'], step[1]['output_bike_resting_hr_continuous'])

In [ ]:
ecg_in = step[0]['input_full_ecg_bike_recovery']
hrr_out = step[1]['output_bike_max_hr_continuous']

In [ ]:
pred = model.predict(ecg_in)

In [ ]:
plt.scatter(hrr_out, pred[0])

In [ ]:
recoveries = []
for i in range(10):
    step = generate_test.__next__()
    recoveries.append(step[1]['output_heart_rate_recovery_ecg_bike_autonomic_function'])